In [2]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.2 MB/s eta 0:00:00


In [3]:
!pip install transformers

In [15]:
def create_llm_prompt(system_prompt, rag_context, user_history, user_query):
    # Format the user's chat history as a string
    history_text = "\n".join(f"Chat {idx + 1}: {entry}" for idx, entry in enumerate(user_history))

    # Assemble the complete prompt using formatted strings
    prompt = (f"System Prompt: {system_prompt}\n\n"
              f"RAG Retrieved Context: {rag_context}\n\n"
              f"User Chat History:\n{history_text}\n\n"
              f"User's Current Query: {user_query}\n")

    return prompt

# Example usage
system_prompt = "Please assist the user with their medical questions."
rag_context = "Disease X is a rare genetic disorder characterized by the absence of symptom Y"
user_history = [
    """{"role": "User", "message": What is disease Z?"},
       {"role": "Assistant", "message": "Disease Z is a rare genetic disorder that affects 1/1,000,000 people."}
    """
]
user_query = "What is disease X?"

# Generate the LLM prompt
prompt = create_llm_prompt(system_prompt, rag_context, user_history, user_query)
print(prompt)


System Prompt: Please assist the user with their medical questions.

RAG Retrieved Context: Disease X is a rare genetic disorder characterized by the absence of symptom Y

User Chat History:
Chat 1: {"role": "User", "message": What is disease Z?"},
       {"role": "Assistant", "message": "Disease Z is a rare genetic disorder that affects 1/1,000,000 people."}
    

User's Current Query: What is disease X?



In [12]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Create a toy document about "Disease X"
documents = ["Disease X is a rare genetic disorder characterized by the absence of symptom Y. It affects 1 in every 100,000 people.",
             "The hippocampus is an area of the brain that is needed to form new memories." ]

# Convert documents to embeddings
embeddings = []
for doc in documents:
    inputs = tokenizer(doc, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    # Use mean pooling to get a single vector representation for each document
    embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

# Create a FAISS index
dimension = embeddings[0].shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(dimension)  # Flat index using L2 distance
# index = faiss.IndexFlatIP(dimension)  # This index computes the inner product


# Add embeddings to index
for emb in embeddings:
    index.add(emb)

In [13]:
def retrieve_document(query):
    # Tokenize and embed the query
    query_inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=512)
    query_outputs = model(**query_inputs)
    query_embedding = query_outputs.last_hidden_state.mean(dim=1).detach().numpy()

    # Search the FAISS index
    D, I = index.search(query_embedding, k=1)  # Search for the top 1 nearest document
    return documents[I[0][0]]

# Example query about Disease X
query = "What is Disease X?"
retrieved_doc = retrieve_document(query)
print(f"Retrieved document: {retrieved_doc}")

Retrieved document: Disease X is a rare genetic disorder characterized by the absence of symptom Y. It affects 1 in every 100,000 people.


In [14]:
query = "How are new memories formed in the brain?"
retrieved_doc = retrieve_document(query)
print(f"Retrieved document: {retrieved_doc}")

Retrieved document: The hippocampus is an area of the brain that is needed to form new memories.
